In [1]:
# ================================================================
# L'AGGREGATORE DOCUMENTALE AVANZATO
# ================================================================
# Regole rispettate:
#   [1] Ingestion eterogenea       -> loader scelto per estensione
#   [2] Context Window Management  -> TokenTextSplitter (token-based)
#   [3] Quality Control            -> soglia similarity_threshold
#   [4] Prompt Dinamico            -> tono + lingua come parametri
#   [5] Pure LCEL                  -> niente create_stuff/create_retrieval
#
# ================================================================


In [2]:

import os
import sys
import shutil
from pathlib import Path
from typing import List

# --- LangChain core ---
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel

# --- Ollama ---
from langchain_ollama import ChatOllama, OllamaEmbeddings

# --- ChromaDB ---
from langchain_chroma import Chroma

# --- Loaders ---
from langchain_community.document_loaders import (
    PyPDFLoader,
    UnstructuredWordDocumentLoader,
    CSVLoader,
    UnstructuredExcelLoader,
)

import os
import sys
import shutil
from pathlib import Path
from typing import List

# --- LangChain core ---
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel

# --- Ollama ---
from langchain_ollama import ChatOllama, OllamaEmbeddings

# --- ChromaDB ---
from langchain_chroma import Chroma

# --- Loaders ---
from langchain_community.document_loaders import (
    PyPDFLoader,
    UnstructuredWordDocumentLoader,
    CSVLoader,
    UnstructuredExcelLoader,
)

from langchain_text_splitters import TokenTextSplitter

c:\Users\Rosy\Desktop\RAG\venv311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DOCUMENTS_PATH   = "./data"
PERSIST_DIRECTORY = "./chroma_db"
RESET_DB         = False                 

EMBEDDING_MODEL  = "llama3"    
LLM_MODEL        = "llama3"


CHUNK_SIZE       = 800
CHUNK_OVERLAP    = 100
SIMILARITY_THRESHOLD = 0.3

### 1. INGESTION  –  Multi-Source Loader manuale

In [4]:
# Mappa: estensione -> classe loader
LOADER_MAP = {
    ".pdf":  PyPDFLoader,
    ".docx": UnstructuredWordDocumentLoader,
    ".csv":  CSVLoader,
    ".xlsx": UnstructuredExcelLoader,
    ".xls":  UnstructuredExcelLoader,
}

In [5]:

def detect_and_load(file_path: str) -> List[Document]:
    """
    Rileva l'estensione del file e invoca il loader corretto.
    Aggiunge metadata 'source_file' con il nome del file.
    """
    ext = Path(file_path).suffix.lower()

    if ext not in LOADER_MAP:
        print(f"  [SKIP] Formato non supportato: {file_path}")
        return []

    loader_cls = LOADER_MAP[ext]
    loader     = loader_cls(file_path)
    docs       = loader.load()

    # Normalizzazione metadata
    for doc in docs:
        doc.metadata["source_file"] = Path(file_path).name

    print(f"  [OK]   {Path(file_path).name} -> {len(docs)} documento/i caricato/i")
    return docs


def load_all_documents() -> List[Document]:
    """
    Scansiona DOCUMENTS_PATH, rileva ogni file, carica con il loader giusto.
    """
    if not os.path.exists(DOCUMENTS_PATH):
        os.makedirs(DOCUMENTS_PATH)
        print(f"[INFO] Creata '{DOCUMENTS_PATH}'. Inserisci i file e rilancia.")
        return []

    print("[INFO] Scansione documenti...")
    all_docs: List[Document] = []

    for file_path in sorted(Path(DOCUMENTS_PATH).iterdir()):
        if file_path.is_file():
            docs = detect_and_load(str(file_path))
            all_docs.extend(docs)

    print(f"[INFO] Totale documenti caricati: {len(all_docs)}")
    return all_docs


### 2. SPLITTING  –  Token-based (unico ammesso)

In [6]:

def chunk_documents(documents: List[Document]) -> List[Document]:
    """
    Divide i documenti in chunk usando TokenTextSplitter.
    """
    if not documents:
        return []

    splitter = TokenTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len,
    )
    chunks = splitter.split_documents(documents)
    print(f"[INFO] Chunk generati: {len(chunks)}")
    return chunks



### 3. VECTORSTORE  –  Sync intelligente

In [7]:

def reset_vectorstore():
    if os.path.exists(PERSIST_DIRECTORY):
        shutil.rmtree(PERSIST_DIRECTORY)
        print("[INFO] Vectorstore resettato.")


def sync_vectorstore(chunks: List[Document], embeddings) -> Chroma:
    """
    Crea o aggiorna il vectorstore.
    Aggiunge solo i chunk ancora mancanti nel DB.
    """
    vectorstore = Chroma(
        collection_name="aggregatore_docs",
        embedding_function=embeddings,
        persist_directory=PERSIST_DIRECTORY,
    )

    if not chunks:
        print("[WARN] Nessun chunk da indicizzare.")
        return vectorstore

    # Confronto con i testi già presenti
    existing  = vectorstore.get()
    existing_texts = set(existing["documents"]) if existing["documents"] else set()
    new_chunks = [c for c in chunks if c.page_content not in existing_texts]

    if new_chunks:
        vectorstore.add_documents(documents=new_chunks)
        print(f"[INFO] Chunk aggiunti al vectorstore: {len(new_chunks)}")
    else:
        print("[INFO] Vectorstore già aggiornato.")

    return vectorstore


###  QUALITY CONTROL

In [8]:

def retrieve_and_filter(vectorstore: Chroma, query_text: str) -> List[Document]:
    """
    Recupera i chunk più simili e scarta quelli sotto SIMILARITY_THRESHOLD.
    Stampa anche un log dei punteggi per trasparenza.
    """
    # Recupera fino a 5 candidati con punteggio
    results_with_scores = vectorstore.similarity_search_with_relevance_scores(
        query=query_text,
        k=5,
    )

    # Log dei punteggi grezzi
    print("\n  [QUALITY CONTROL] Punteggi recuperati:")
    for doc, score in results_with_scores:
        src  = doc.metadata.get("source_file", "?")
        flag = "✓" if score >= SIMILARITY_THRESHOLD else "✗"
        print(f"    {flag} score={score:.3f}  [{src}]")

    # Filtra: tieni solo quelli sopra la soglia
    filtered = [doc for doc, score in results_with_scores if score >= SIMILARITY_THRESHOLD]

    print(f"  [QUALITY CONTROL] Chunk passati il filtro: {len(filtered)}/{len(results_with_scores)}\n")
    return filtered



### 4. PROMPT PARAMETRICO

### Parametri nel prompt: 
 <br>  {context}  -> chunk recuperati (iniettati dalla chain)
 <br>    {input}    -> domanda dell'utente
 <br>   {tone}     -> tono della risposta  (es. professionale, amichevole, tecnico)
 <br>    {lingua}   -> lingua della risposta (es. italiano, english, español)


In [9]:
SYSTEM_PROMPT = (
    "Sei un aggregatore documentale esperto. Il tuo compito è rispondere "
    "usando SOLO le informazioni presenti nel contesto fornito.\n"
    "Se il contesto non contiene la risposta, dilo esplicitamente.\n\n"
    "--- CONTESTO RECUPERATO ---\n"
    "{context}\n"
    "--- FINE CONTESTO ---\n\n"
    "Rispondi nel tono specificato e nella lingua richiesta.\n"
    "Tono:   {tone}\n"
    "Lingua: {lingua}\n"
)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "Domanda: {input}"),
])

### 5. PURE LCEL

In [10]:
#
# Struttura della pipeline:
#
#   input (dict)
#     |
#     ├── "input"  ──────────────────────────> retriever ──> format_docs ──> "context"
#     |                                                                        |
#     ├── "tone"   ──────────────────────────────────────────────────────>     |
#     |                                                                        v
#     ├── "lingua" ──────────────────────────────────────────> prompt_template (assembla)
#     |                                                                        |
#     └── "input"  ──────────────────────────────────────────>                 v
#                                                              llm ──> AIMessage
#                                                                        |
#                                                                        v
#                                                              parse_output (stringa)
#

In [11]:

def format_docs(docs: List[Document]) -> str:
    """
    Formatta i Document recuperati in una stringa pulita per il prompt.
    Aggiunge anche il nome del file sorgente per trasparenza.
    """
    if not docs:
        return "[Nessun documento rilevante trovato nella knowledge base.]"

    parts = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get("source_file", "sconosciuto")
        parts.append(f"[Documento {i} — {source}]\n{doc.page_content}")

    return "\n\n".join(parts)


def build_lcel_chain(vectorstore: Chroma, llm):
    """
    Costruisce la chain RAG in pure LCEL.
    Il recupero usa retrieve_and_filter (quality control manuale).
    """

    # Step 1: recupera i chunk con filtro e li formatta
    retrieval_step = (
        RunnableLambda(lambda x: retrieve_and_filter(vectorstore, x["input"]))
        | RunnableLambda(format_docs)
    )

    # Step 2: costruisce il dizionario che va nel prompt
    # RunnableParallel esegue in parallelo i vari branch
    chain_input = RunnableParallel(
        context  = retrieval_step,                          # recupera + formatta
        input    = RunnablePassthrough() | RunnableLambda(lambda x: x["input"]),
        tone     = RunnablePassthrough() | RunnableLambda(lambda x: x.get("tone", "professionale")),
        lingua   = RunnablePassthrough() | RunnableLambda(lambda x: x.get("lingua", "italiano")),
    )

    # Step 3: assembla prompt -> llm -> estrai testo
    generation_step = prompt_template | llm | RunnableLambda(lambda msg: msg.content)

    # Step 4: composizione finale
    full_chain = chain_input | generation_step

    return full_chain


In [12]:

def query(chain, question: str, tone: str = "professionale", lingua: str = "italiano") -> str:
    """
    Esegue una query sulla chain LCEL.
    """
    try:
        return chain.invoke({
            "input":  question,
            "tone":   tone,
            "lingua": lingua,
        })
    except Exception as e:
        return f"[ERRORE] {e}"


In [13]:

if __name__ == "__main__":

    # --- Reset opzionale ---
    if RESET_DB:
        reset_vectorstore()

    # --- Embeddings (nomic-embed-text via Ollama) ---
    print("[INFO] Inizializzazione embeddings (nomic-embed-text)...")
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

    # --- Caricamento multi-source ---
    documents = load_all_documents()

    # --- Chunking token-based ---
    chunks = chunk_documents(documents)

    # --- Sync vectorstore ---
    print("[INFO] Sincronizzazione vectorstore...")
    vectorstore = sync_vectorstore(chunks, embeddings)

    # --- LLM ---
    llm = ChatOllama(model=LLM_MODEL, temperature=0)

    # --- Chain LCEL ---
    print("[INFO] Costruzione chain LCEL...")
    rag_chain = build_lcel_chain(vectorstore, llm)

    # ============================================================
    # QUERY DI ESEMPIO
    # ============================================================

    print("\n" + "=" * 60)
    print(" QUERY 1 – Sintesi (italiano, tono amichevole)")
    print("=" * 60)
    print(query(rag_chain,
                question="Fammi una sintesi dei documenti disponibili.",
                tone="amichevole",
                lingua="italiano"))

    print("\n" + "=" * 60)
    print(" QUERY 2 – Dettaglio tecnico (italiano, tono tecnico)")
    print("=" * 60)
    print(query(rag_chain,
                question="Quali sono i punti principali del documento nanotech1.pdf?",
                tone="tecnico",
                lingua="italiano"))

    print("\n" + "=" * 60)
    print(" QUERY 3 – Stessa domanda ma in inglese")
    print("=" * 60)
    print(query(rag_chain,
                question="Summarize the content of cell1.docx.",
                tone="professional",
                lingua="english"))


[INFO] Inizializzazione embeddings (nomic-embed-text)...
[INFO] Scansione documenti...
  [OK]   nanotech1.pdf -> 3 documento/i caricato/i
[INFO] Totale documenti caricati: 3
[INFO] Chunk generati: 5
[INFO] Sincronizzazione vectorstore...
[INFO] Chunk aggiunti al vectorstore: 5
[INFO] Costruzione chain LCEL...

 QUERY 1 – Sintesi (italiano, tono amichevole)


C:\Users\Rosy\AppData\Local\Temp\ipykernel_9324\522596172.py:7: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='5fd789c0-64a0-4906-9ca8-d87c7c572197', metadata={'creationdate': '2026-02-02T17:56:42+01:00', 'source_file': 'nanotech1.pdf', 'author': 'Rosy', 'source': 'data\\nanotech1.pdf', 'total_pages': 3, 'page_label': '3', 'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'moddate': '2026-02-02T17:56:42+01:00', 'page': 2}, page_content='anche qualche limite, come la possibile tossicità e il rischio di accumulo nel fegato, specialmente se \ncontengono elementi pesanti come il cadmio.'), 0.6340295577795377), (Document(id='9110e46d-a08b-4d37-89b1-5de9a7b2e675', metadata={'creator': 'Microsoft® Word 2019', 'moddate': '2026-02-02T17:56:42+01:00', 'page': 1, 'creationdate': '2026-02-02T17:56:42+01:00', 'source': 'data\\nanotech1.pdf', 'source_file': 'nanotech1.pdf', 'producer': 'Microsoft® Word 2019', 'page_label': '2', 'author': 'Rosy', 'total


  [QUALITY CONTROL] Punteggi recuperati:
    ✓ score=0.634  [nanotech1.pdf]
    ✗ score=0.057  [nanotech1.pdf]
    ✗ score=-0.129  [nanotech1.pdf]
    ✗ score=-0.203  [nanotech1.pdf]
    ✗ score=-0.333  [nanotech1.pdf]
  [QUALITY CONTROL] Chunk passati il filtro: 1/5

Sembra che il documento "nanotech1.pdf" parli di nanotecnologie! In particolare, menziona alcuni limiti relativi all'utilizzo di queste tecnologie, come ad esempio la possibile tossicità e il rischio di accumulo nel fegato, specialmente se contengono elementi pesanti come il cadmio.

 QUERY 2 – Dettaglio tecnico (italiano, tono tecnico)


C:\Users\Rosy\AppData\Local\Temp\ipykernel_9324\522596172.py:7: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='5fd789c0-64a0-4906-9ca8-d87c7c572197', metadata={'source': 'data\\nanotech1.pdf', 'author': 'Rosy', 'page_label': '3', 'creationdate': '2026-02-02T17:56:42+01:00', 'producer': 'Microsoft® Word 2019', 'page': 2, 'total_pages': 3, 'source_file': 'nanotech1.pdf', 'moddate': '2026-02-02T17:56:42+01:00', 'creator': 'Microsoft® Word 2019'}, page_content='anche qualche limite, come la possibile tossicità e il rischio di accumulo nel fegato, specialmente se \ncontengono elementi pesanti come il cadmio.'), 0.5014556764110047), (Document(id='9110e46d-a08b-4d37-89b1-5de9a7b2e675', metadata={'author': 'Rosy', 'source': 'data\\nanotech1.pdf', 'source_file': 'nanotech1.pdf', 'moddate': '2026-02-02T17:56:42+01:00', 'page_label': '2', 'producer': 'Microsoft® Word 2019', 'total_pages': 3, 'creator': 'Microsoft® Word 2019', 'page': 1, 'creationdate': '2026-02-02T17:56


  [QUALITY CONTROL] Punteggi recuperati:
    ✓ score=0.501  [nanotech1.pdf]
    ✗ score=0.070  [nanotech1.pdf]
    ✗ score=-0.094  [nanotech1.pdf]
    ✗ score=-0.191  [nanotech1.pdf]
    ✗ score=-0.306  [nanotech1.pdf]
  [QUALITY CONTROL] Chunk passati il filtro: 1/5

Secondo il contesto, i punti principali del documento nanotech1.pdf sembrano essere la discussione sulla tossicità e sul rischio di accumulo nel fegato dei materiali nanotecnologici, specialmente quelli che contengono elementi pesanti come il cadmio.

 QUERY 3 – Stessa domanda ma in inglese


C:\Users\Rosy\AppData\Local\Temp\ipykernel_9324\522596172.py:7: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='5fd789c0-64a0-4906-9ca8-d87c7c572197', metadata={'author': 'Rosy', 'page': 2, 'creator': 'Microsoft® Word 2019', 'total_pages': 3, 'moddate': '2026-02-02T17:56:42+01:00', 'source': 'data\\nanotech1.pdf', 'creationdate': '2026-02-02T17:56:42+01:00', 'producer': 'Microsoft® Word 2019', 'source_file': 'nanotech1.pdf', 'page_label': '3'}, page_content='anche qualche limite, come la possibile tossicità e il rischio di accumulo nel fegato, specialmente se \ncontengono elementi pesanti come il cadmio.'), 0.2871977465867813), (Document(id='9110e46d-a08b-4d37-89b1-5de9a7b2e675', metadata={'author': 'Rosy', 'source': 'data\\nanotech1.pdf', 'page': 1, 'page_label': '2', 'creator': 'Microsoft® Word 2019', 'source_file': 'nanotech1.pdf', 'moddate': '2026-02-02T17:56:42+01:00', 'producer': 'Microsoft® Word 2019', 'total_pages': 3, 'creationdate': '2026-02-02T17:56


  [QUALITY CONTROL] Punteggi recuperati:
    ✗ score=0.287  [nanotech1.pdf]
    ✗ score=-0.137  [nanotech1.pdf]
    ✗ score=-0.322  [nanotech1.pdf]
    ✗ score=-0.353  [nanotech1.pdf]
    ✗ score=-0.400  [nanotech1.pdf]
  [QUALITY CONTROL] Chunk passati il filtro: 0/5

I apologize, but there is no document or information available to summarize. The knowledge base does not contain any relevant documents or data.


In [15]:
# ================================================================
# AGGREGATORE DOCUMENTALE AVANZATO – STREAMLIT + LCEL
# ================================================================

import streamlit as st
from pathlib import Path
from typing import List


# ================================================================
# HEADER STREAMLIT
# ================================================================
st.set_page_config(page_title="Aggregatore Documentale Avanzato", layout="wide")
st.title("📚 Aggregatore Documentale Avanzato")
st.markdown(
    """
    Inserisci una domanda, scegli il tono e la lingua della risposta.
    L'aggregatore cercherà tra i documenti caricati nella knowledge base.
    """
)

# ================================================================
# INPUT UTENTE
# ================================================================
user_query = st.text_area("✏️ Domanda:", height=100)
user_tone = st.selectbox("🎨 Tono della risposta:", ["professionale", "amichevole", "tecnico"])
user_lang = st.selectbox("🌐 Lingua della risposta:", ["italiano", "english", "español"])

# ================================================================
# BOTTONI
# ================================================================
if st.button("Genera Risposta"):
    with st.spinner("Caricamento e generazione risposta..."):
        # --- Reset opzionale vectorstore ---
        if RESET_DB:
            reset_vectorstore()

        # --- Embeddings ---
        embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

        # --- Caricamento documenti ---
        documents = load_all_documents()
        if not documents:
            st.warning("📂 Nessun documento trovato. Inserisci file in ./data")
        else:
            # --- Chunking ---
            chunks = chunk_documents(documents)

            # --- Sync vectorstore ---
            vectorstore = sync_vectorstore(chunks, embeddings)

            # --- LLM ---
            llm = ChatOllama(model=LLM_MODEL, temperature=0)

            # --- Chain LCEL ---
            rag_chain = build_lcel_chain(vectorstore, llm)

            # --- Query ---
            risposta = query(rag_chain, user_query, tone=user_tone, lingua=user_lang)
            st.markdown("### ✅ Risposta generata:")
            st.write(risposta)

# ================================================================
# FOOTER
# ================================================================
st.markdown("---")
st.markdown("Aggregatore Documentale Avanzato – LCEL + LangChain + Ollama + ChromaDB")


2026-02-05 15:17:21.927 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 15:17:21.929 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 15:17:22.254 
  command:

    streamlit run c:\Users\Rosy\Desktop\RAG\venv311\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-02-05 15:17:22.256 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 15:17:22.258 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 15:17:22.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 15:17:22.262 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored 

DeltaGenerator()